In [ ]:
#Self-hyperparam selection: https://link.springer.com/article/10.1007/s11063-024-11578-0
#Self-pruning: https://github.com/skarifahmed/seMLP/blob/main/src/Prune.py

In [41]:
from utils import Utils
from color import color 
import pandas as pd
import numpy as np
import os
# libraries
import joblib

# scale features
from sklearn import preprocessing
from sklearn import impute
# classifier
from sklearn.ensemble import ExtraTreesClassifier
# scoring metrics
from sklearn.metrics import confusion_matrix, matthews_corrcoef

# custom scripts
import sys
sys.path.insert(0, "%s" % "CV/")
from sklearn.model_selection import train_test_split, GridSearchCV, GroupShuffleSplit, StratifiedShuffleSplit, cross_validate, StratifiedKFold
from sklearn.metrics import roc_curve, auc, recall_score, accuracy_score, precision_score, confusion_matrix, make_scorer, matthews_corrcoef, jaccard_score

In [8]:
site_path = "/Users/sanjanayasna/csc334/MLP_MAHOMES/sites_calculated_features.txt"

In [11]:
#read in feature set:
sites = pd.read_csv(site_path)
sites = sites.set_index('SITE_ID',drop=True)

# The following labels need to be changed after looking over literature (see Feehan, Franklin, Slusky 2021)
change_site_labels = ["5zb8_0", "6aci_0", "6oq7_0", "6pjv_1", "6q55_0",
                      "6q55_2", "6rmg_0", "6rtg_0", "6rw0_0", "6v77_0"]

# The following sites are removed due to unkopwn correct labels (see Feehan, Franklin, Slusky 2021)
sites.loc[sites.index.isin(change_site_labels), 'Catalytic']=True
remove_sites = ["6mf0_1", "6okh_0", "6qwo_0", "6r9n_0"]
sites=sites.loc[~sites.index.isin(remove_sites)]

#print shape of dataset
print(color.BOLD + "All features:" + color.END)
print("sites: %s \tcolumns: %s"%(sites.shape[0], sites.shape[1]))
sizes = sites.groupby(["Set", "Catalytic"]).size()
print(sizes)

All features:
sites: 3981 	columns: 485
Set   Catalytic
data  False        2636
      True          829
test  False         345
      True          171
dtype: int64


In [37]:
#save_models toggel
save_models = False
#pkl output path
pkl_out = None

In [22]:
sites.head()

,Catalytic,MetalCodes,MetalAtoms,fa_atr_Sum_3.5,fa_rep_Sum_3.5,fa_sol_Sum_3.5,fa_intra_atr_xover4_Sum_3.5,fa_intra_rep_xover4_Sum_3.5,fa_intra_sol_xover4_Sum_3.5,lk_ball_Sum_3.5,...,geom_cn8,geom_cn9,geom_Filled,geom_PartFilled,geom_AvgN,geom_AvgO,geom_AvgS,geom_AvgOther,SC_vol_perc,Set
SITE_ID,,,,,,,,,,,,,,,,,,,,,
6s9z_0,True,1,1,-33.20757,20.22373,26.34441,-1.88617,0.46054,2.14096,14.05052,...,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.910384,test
6g5l_0,True,1,1,-27.04899,39.17134,22.76555,-1.71942,0.45999,2.05517,12.94894,...,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.862189,test
6hwz_0,True,1,1,-27.30433,35.04867,23.45195,-1.62146,0.35902,1.91231,13.06378,...,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.991431,test
6qww_0,True,1,1,-25.36664,12.54178,27.17902,-1.14349,0.22087,1.68091,11.47631,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.864546,test
6qww_1,False,1,1,-30.53159,8.99318,27.77842,-1.00782,0.39657,1.04229,13.23736,...,0.0,0.0,1.0,0.0,0.0,4.0,0.0,1.0,0.990893,test


In [ ]:
#Get scaled features
data_scaled, Tsites_scaled = Utils.get_scaled_features(sites =sites, pkl_out=pkl_out, save_models=save_models)
#Print stats
print(color.BOLD + "All scaled data-set features:" + color.END)
print("sites: %s \tcolumns: %s"%(data_scaled.shape[0], data_scaled.shape[1]))ß
print(data_scaled.groupby(["Catalytic"]).size())

print(color.BOLD + "\nAll scaled T-metal-site features:" + color.END)
print("sites: %s \tcolumns: %s"%(Tsites_scaled.shape[0], Tsites_scaled.shape[1]))
print(Tsites_scaled.groupby(["Catalytic"]).size())

In [45]:
dir = "/Users/sanjanayasna/csc334/MLP_MAHOMES/data/"
#save the scaled data
data_scaled.to_csv(os.path.join(dir, "data_scaled.csv"))
Tsites_scaled.to_csv(os.path.join(dir, "Tsites_scaled.csv"))